# Understanding the Out-of-Sample Error and the No Free Lunch Theorem

In this notebook, we will walk through the mathematics behind out-of-sample error and the No Free Lunch (NFL) Theorem. We will break down important equations and explore examples using Python to reinforce key concepts.


## Out-of-Sample Error (Equation 1.1)

The out-of-sample error for a learning algorithm 
$$\mathcal{L}_a$$ 
is given by:

$$
E_{\text{ote}}(\mathcal{L}_a | X, f) = \sum_{h} \sum_{x \in \mathcal{X} - X} P(x) \mathbb{I}(h(x) \neq f(x)) P(h | X, \mathcal{L}_a)
$$

Where:
- $
E_{\text{ote}}(\mathcal{L}_a | X, f) 
$ Is the out-of-sample error for a learning algorithm, given the training set and the ground truth.

- $
\mathcal{X} - X
$ Is the whole input space subtract the training set, which is therefore equivalent to all unseen data (i.e the test set).

- $
P(x)
$ Is the probability of observing (unseen/test) sample $x$.

- $
\mathbb{I}(h(x) \neq f(x))
$ Is the indicator function, which will be $0$ when our hypothesis (i.e. prediction) for unseen sample $x$ is equal to the ground truth, and $1$ otherwise.

- $
P(h | X, \mathcal{L}_a)
$ Is the probability of obtaining the hypothesis $h$ given the training data $X$ and the learning algorithm $\mathcal{L}_a$.